In [ ]:
#Remove duplicates withous using set
# 
l = [1,2,7,6,8,6,2,8,6,4,2]

s = []

for i in l:
    if i not in s:
        s.append(i)
    
print(s)

[1, 2, 7, 6, 8, 4]


In [ ]:
#Remove vowels from below words without splitting the word
# s = "environment"
text = "the Environment"
vowels = "aeiouAEIOU"

result = "".join(char for char in text if char not in vowels)
print(result)





In [ ]:
def remove_vowels(s):
    vowels = "aeiouAEIOU"
    return "".join(c for c in s if c not in vowels)

print(remove_vowels("the Environment"))


In [ ]:
text = "the Environment"
vowels = "aeiouAEIOU"
result = ""

for ch in text:
    if ch not in vowels:
        result += ch

print(result)


In [ ]:
import time
import functools
import random
from openai import OpenAI
from openai import APIError, RateLimitError, APITimeoutError

client = OpenAI()

def retry_with_backoff(retries=5, backoff_in_seconds=1, max_backoff=60):
    """
    Decorator for retrying an OpenAI API call using exponential backoff.
    Retries on RateLimitError, APIError, and APITimeout.
    """
    def decorator(function):
        @functools.wraps(function)
        def wrapper(*args, **kwargs):
            delay = backoff_in_seconds

            for attempt in range(retries):
                try:
                    return function(*args, **kwargs)

                except (RateLimitError, APITimeoutError, APIError) as e:
                    # Last attempt: re-raise
                    if attempt == retries - 1:
                        print(f"[FAILED] Retries exhausted. Raising error.")
                        raise

                    print(
                        f"[Retry {attempt+1}/{retries}] "
                        f"Error: {e}. Retrying in {delay:.2f}s..."
                    )

                    # Sleep before retry
                    time.sleep(delay)

                    # Exponential backoff with jitter
                    delay = min(delay * 2 + random.random(), max_backoff)

        return wrapper
    return decorator
@retry_with_backoff(retries=5, backoff_in_seconds=1)
def call_openai(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message["content"]


# Example usage
if __name__ == "__main__":
    print(call_openai("Explain exponential backoff in simple terms."))